In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Loading Data
We will use the white wine quality that you have used previously. The data is downloaded to your project from the UCI Machine Learning Repository using the following code:

In [4]:
data = pd.read_csv('wine_data.csv',sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [74]:
x = data.iloc[:,:-1]
y= data.iloc[:,-1]

In [75]:
scaler = StandardScaler()
x = scaler.fit_transform(x)


In [76]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.3,random_state=20)

In [77]:
input_shape = x_train.shape[1]
input_shape

11

In [120]:
small_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64,activation='relu',input_shape=(input_shape,)),
    tf.keras.layers.Dense(1)
])
small_model.compile(loss='mse',optimizer='adam')

In [121]:
def input_fn(features,label,training=True,batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((features,label))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

## Model to Estimator

In [122]:
keras_small_estimator = tf.keras.estimator.model_to_estimator(keras_model=small_model,model_dir='keras_small_classifier')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'keras_small_classifier', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [123]:
keras_small_estimator.train(input_fn=lambda:input_fn(x_train,y_train,training=True),steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='keras_small_classifier/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: keras_small_classifier/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 4 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into keras_small_classifier/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 31.929977, step = 0
INFO:tensorflow:global_step/sec: 831.614
INFO:tensorflow:loss = 11.903

In [124]:
result = keras_small_estimator.evaluate(input_fn=lambda:input_fn(x_test,y_test,training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-04-18T08:44:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from keras_small_classifier/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.17704s
INFO:tensorflow:Finished evaluation at 2023-04-18-08:44:02
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 0.5144854
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: keras_small_classifier/model.ckpt-5000


In [125]:
print(f'Eval Result {result}')

Eval Result {'loss': 0.5144854, 'global_step': 5000}


In [128]:
%load_ext tensorboard
%tensorboard --logdir ./keras_small_classifier

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [127]:
!kill 59449